In [1]:
import pandas as pd
class DataHandler:
    """
        Get data from GSC Bucket 
    """
    def __init__(self):
        self.csvfile1 = None
        self.csvfile2 = None
        self.gouped_data = None
    def get_data(self):
        print(" - - - fetch data from gcs bucket : - - - ")
        listing = pd.read_csv('https://storage.googleapis.com/h3-data/listings_final.csv', sep=';')
        prices = pd.read_csv('https://storage.googleapis.com/h3-data/price_availability.csv', sep=';')
        self.csvfile1, self.csvfile2 = listing, prices  
        return " - - - data loaded - - - \nFiles : \n  - listing {} \n  - prices {}".format(listing.shape,prices.shape)
    def group_data(self):
        data = self.csvfile2.groupby('listing_id')['local_price'].mean()
        self.gouped_data = pd.merge(data, self.csvfile1, on='listing_id')
        print(" - - - data merged - - - ")
    def get_process_data(self):
        self.get_data()
        self.group_data()
        print(" - - - data processed - - - ")

In [2]:
data = DataHandler()
data.get_process_data()
%time 

 - - - fetch data from gcs bucket : - - - 
 - - - data merged - - - 
 - - - data processed - - - 
CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.96 µs


In [18]:
import logging
class FeatureRecipe(DataHandler):
    """
    Feature processing class
    """
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.category = None
        self.discrete_variable = None
        self.continous_variable = None
        
    #Useless feature
    def drop_useless(self):
        """
        Drop useless column
        """

        def drop_specific_col(self):
            dropped_sepcific_col = []
            dropped_sepcific_col.append('name')
            dropped_sepcific_col.append('Unnamed: 0')
            logging.debug("name and Unamed: 0 columns are useless")
            return dropped_sepcific_col
               
        def drop_nan_col(self):
            dropped_nan_col = [] 
            for (columnName, columnData) in self.data.iteritems(): 
                if(self.data[columnName].isna().all() == True):
                    dropped_nan_col.append(columnName)
            logging.debug("{} feature have 100% NaN ".format(len(dropped_nan_col)))
            return dropped_nan_col
                
        self.data = self.data.drop(drop_specific_col(self), axis=1)
        self.data = self.data.drop(drop_nan_col(self), axis=1)
        print("- - - drop useless columns - - - ")
        
    #def deal_duplicate(self):
        #def drop_duplicate_col(self):
            #duplicated_col = self.data.loc[:,~self.data.columns.duplicated()]
            #logging.debug("{} feature are duplicated ".format(len(duplicated_col)))
            #return duplicated_col
        #self.data = drop_duplicate_col(self)
        #print("- - - drop duplicated column - - - ")
        
    def deal_duplicate(self):
        duplicated = []
        def get_duplicated(self):
            rows = []
            for c in self.data.columns.to_list():
                row = []
                for l in self.data[c]:
                    row.append(l)
                rows.append(row)
            
            for i in range(0, len(rows)):    
                for j in range(i+1, len(rows)):    
                    if(rows[i] == rows[j]):
                        duplicated.append(j);               
            return duplicated
        
        self.data = self.data.drop(get_duplicated(self))
        logging.debug('{} duplicated features droped'.format(len(get_duplicated(self))))
                      
    #Nan
    def drop_nanp(self, thresold: float):
        def deal_nanp(df:pd.DataFrame, thresold: float):
            bf=[]
            for c in self.data.columns.to_list():
                if self.data[c].isna().sum()/self.data.shape[0] > thresold:
                    bf.append(c)
            logging.debug("{} feature have more than {} NaN ".format(len(bf), thresold))
            logging.debug('\n\n - - - features - - -  \n {}'.format(bf))
            return bf 
        self.data = self.data.drop(deal_nanp(self.data, thresold), axis=1)
        logging.debug('Some NaN features droped according to {} thresold'.format(thresold))
            
    #Separate feature for feature engineering
    def separate_variable_types(self) -> None:
        """
        Sépare les colonnes en catégorie, variable discrète et variable continue
        """
        def get_discrete_variable(self):
            self.data.update(self.data.select_dtypes(include='bool').astype('int64'))
            return self.data.select_dtypes(include='int64')
            
        def get_continous_variable(self):
            return self.data.select_dtypes(include='float64')
            
        def get_category(self):
            return self.data.select_dtypes(include='object')
            
        self.discrete_variable = get_discrete_variable(self)
        self.continous_variable = get_continous_variable(self)
        self.category = get_category(self)
        print("- - - separate variable types - - - ")

    #Datetime
    def deal_dtime(self):
        pass
    
    def prepare_data(self, thresold: float):
        """
        Wrap code above
        """
        self.drop_useless()
        self.deal_duplicate()
        self.drop_nanp(thresold)
        self.separate_variable_types()
        self.deal_dtime()
        print("- - - data processed - - -")

In [19]:
recipe = FeatureRecipe(data.gouped_data)
recipe.prepare_data(0.06)
%time 

- - - drop useless columns - - - 
- - - separate variable types - - - 
- - - data processed - - -
CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.44 µs


In [46]:
from sklearn.model_selection import train_test_split
class FeatureExtractor:
    """
    Feature Extractor class
    """    
    def __init__(self, data: pd.DataFrame, flist: list):
        """
            Input : pandas.DataFrame, feature list to drop
            Output : X_train, X_test, y_train, y_test according to sklearn.model_selection.train_test_split
        """
        
        self.data = data
        self.flist = flist
        self.X_train, self.X_test, self.y_train, self.y_test = None, None, None, None
    
    def extract(self):
        """
            drop useless column and set x and y
        """
        self.data = self.data.drop(self.flist, axis=1)
        x = self.data[["person_capacity", "beds","bedrooms", "bathrooms"]]
        y = self.data["local_price"]
        return x, y
    
    def split(self, size: float):
        """
            train test split
        """
        x, y = self.extract()
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(x, y, random_state=42,test_size=size)
        return self.X_train, self.X_test, self.y_train, self.y_test
        
    def train(self, size: float):
        self.split(size)

In [47]:
Fextractor = FeatureExtractor(recipe.data, ['listing_id', 'type', 'city', 'latitude', 'longitude', 'is_rebookable', 'is_new_listing', 'is_fully_refundable', 'is_host_highly_rated', 'is_business_travel_ready', 'pricing_weekly_factor', 'pricing_monthly_factor'])
Fextractor.train(0.30)
%time 

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.63 µs
